In [23]:
from utils.gaussian_viewing_utils import * 
import matplotlib.pyplot as plt
import os
from moviepy.editor import ImageSequenceClip
from gaussian_renderer import render, GaussianModel
from scene.cameras import Camera as GSCamera
from scene.dataset_readers import readRNNSIMSceneInfo
import kaolin
from scene import Scene
import time

import argparse

def log_tensor(t, name, **kwargs):
    print(kaolin.utils.testing.tensor_info(t, name=name, **kwargs))

In [24]:
# can come up with a list of model_paths

In [25]:
model_path = '/home/sierra/gaussian-splatting/eval_depth_norm2/cecum'
#model_path = '/home/sierra/gaussian-splatting/eval/invivo57'

# calculate number of views should be length of cameras.json
cam_path = os.path.join(model_path, 'cameras.json')
with open(cam_path, 'r') as f:
    cameras = json.load(f)
num_views = len(cameras)

# Camera ID
novel_view_camera_id = 40   

# if visualization folder doesn't exist, make it
if not os.path.exists(model_path + '/visualization'):
    os.makedirs(model_path + '/visualization')

# Novel view visualization output
nv_output_path = model_path + f'/visualization/novel-views-around-camera-id{novel_view_camera_id}_xaxis.mp4'

In [26]:
class PipelineParamsNoparse:
    """ Same as PipelineParams but without argument parser. """
    def __init__(self):
        self.convert_SHs_python = False
        self.compute_cov3D_python = False                      
        self.debug = False

        
def load_checkpoint(model_path, sh_degree=3, iteration=-1):
    # Find checkpoint
    checkpt_dir = os.path.join(model_path, "point_cloud")
    if iteration == -1:
        iteration = searchForMaxIteration(checkpt_dir)
    checkpt_path = os.path.join(checkpt_dir, f"iteration_{iteration}", "point_cloud.ply")
    
    # Load guassians
    gaussians = GaussianModel(sh_degree)
    gaussians.load_ply(checkpt_path)                                                 
    return gaussians


def try_load_camera(model_path, idx=0):
    """ Load one of the default cameras for the scene. """
    cam_path = os.path.join(model_path, 'cameras.json')
    if not os.path.exists(cam_path):
        print(f'Could not find saved cameras for the scene at {cam_path}; using default for ficus.')
        return GSCamera(colmap_id=0,
                        R=np.array([[-9.9037e-01,  2.3305e-02, -1.3640e-01], [ 1.3838e-01,  1.6679e-01, -9.7623e-01], [-1.6444e-09, -9.8571e-01, -1.6841e-01]]), 
                        T=np.array([6.8159e-09, 2.0721e-10, 4.03112e+00]), 
                        FoVx=0.69111120, FoVy=0.69111120, 
                        image=torch.zeros((3, 800, 800)),  # fake 
                        gt_alpha_mask=None, image_name='fake', uid=0)
        
    with open(cam_path) as f:
        data = json.load(f)
        raw_camera = data[idx]
        
    tmp = np.zeros((4, 4))
    tmp[:3, :3] = raw_camera['rotation']
    tmp[:3, 3] = raw_camera['position']
    tmp[3, 3] = 1
    name = raw_camera['img_name']
    C2W = np.linalg.inv(tmp)
    R = C2W[:3, :3].transpose()
    T = C2W[:3, 3]
    width = raw_camera['width']
    height = raw_camera['height']
    fovx = focal2fov(raw_camera['fx'], width)
    fovy = focal2fov(raw_camera['fy'], height)
    return GSCamera(colmap_id=0,
                    R=R, T=T, FoVx=fovx, FoVy=fovy, 
                    image=torch.zeros((3, height, width)),  # fake 
                    depth=torch.zeros((height, width)),  # fake
                    gt_alpha_mask=None, image_name=name, uid=0)

def load_camera_map(camera_file, name_convention='id.png'):
    with open(camera_file, 'r') as f:
        data = json.load(f)
    if name_convention == 'id.png':
        return {int(cam['img_name']): cam['id'] for cam in data}
    elif name_convention == 'id_color.jpg':
        return {int(cam['img_name'].split('_')[0]): cam['id_color'] for cam in data}
    else:
        raise ValueError('name_convention not recognized')

gaussians = load_checkpoint(model_path)
pipeline = PipelineParamsNoparse()
background = torch.tensor([0, 0, 0], dtype=torch.float32, device="cuda")
center_camera = try_load_camera(model_path, idx=novel_view_camera_id)

args = argparse.Namespace()
args.resolution = 1
args.model_path = model_path
args.source_path = dataset_path
args.images = dataset_path + '/images'
args.depths = dataset_path + '/depths'
args.eval = True
args.data_device = 'cuda'

scene = Scene(args, gaussians)

In [27]:
!export CUDA_VISIBLE_DEVICES=0

In [28]:
!nvidia-smi

Thu Mar  7 11:16:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.07             Driver Version: 535.161.07   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               Off | 00000000:41:00.0 Off |                  Off |
| 30%   56C    P2              61W / 300W |   7321MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [29]:
render_rotate_about_x_path_video(model_path, novel_view_camera_id, 30, 30, nv_output_path, num_steps=200, num_repeats=1)

Moviepy - Building video /home/sierra/gaussian-splatting/eval_depth_norm2/cecum/visualization/novel-views-around-camera-id40_xaxis.mp4.
Moviepy - Writing video /home/sierra/gaussian-splatting/eval_depth_norm2/cecum/visualization/novel-views-around-camera-id40_xaxis.mp4



Moviepy - Done !
Moviepy - video ready /home/sierra/gaussian-splatting/eval_depth_norm2/cecum/visualization/novel-views-around-camera-id40_xaxis.mp4
Image Video saved to /home/sierra/gaussian-splatting/eval_depth_norm2/cecum/visualization/novel-views-around-camera-id40_xaxis.mp4
Moviepy - Building video /home/sierra/gaussian-splatting/eval_depth_norm2/cecum/visualization/novel-views-around-camera-id40_xaxis_depth.mp4.
Moviepy - Writing video /home/sierra/gaussian-splatting/eval_depth_norm2/cecum/visualization/novel-views-around-camera-id40_xaxis_depth.mp4



Moviepy - Done !
Moviepy - video ready /home/sierra/gaussian-splatting/eval_depth_norm2/cecum/visualization/novel-views-around-camera-id40_xaxis_depth.mp4
Depth Video saved to /home/sierra/gaussian-splatting/eval_depth_norm2/cecum/visualization/novel-views-around-camera-id40_xaxis_depth.mp4


In [5]:
images = []
depth_images = []
# find global min and global max of depths 
global_min = 100000
global_max = -100000

train_idx = 0
test_idx = 0
for idx in range(num_views):
    if idx % 8 != 0:
        image_file = model_path + '/train/ours_7000/renders/' + str(train_idx).zfill(5) + '.png'
        depth_file = model_path + '/train/ours_7000/renders/' + str(train_idx).zfill(5) + '_depth.png'
        train_idx += 1
    else:
        image_file = model_path + '/test/ours_7000/renders/' + str(test_idx).zfill(5) + '.png'
        depth_file = model_path + '/test/ours_7000/renders/' + str(test_idx).zfill(5) + '_depth.png'
        test_idx += 1
    image = plt.imread(image_file)
    images.append(image*255)
    depth_image = plt.imread(depth_file)
    if depth_image.min() < global_min:
        global_min = depth_image.min()
    if depth_image.max() > global_max:
        global_max = depth_image.max()
    depth_images.append(depth_image)

In [6]:
# print global min and max
print(global_min, global_max)

0.12156863 1.0


In [7]:
# all depths should be mapped from global_min global_max to 0 255
for idx, depth_image in enumerate(depth_images):
    depth_images[idx] = (depth_image - global_min) / (global_max - global_min) * 255

In [8]:
output_path = model_path + '/visualization/allcameras.mp4'
depth_path = model_path + '/visualization/allcameras_depth.mp4'

In [9]:
clip = ImageSequenceClip(list(images), fps=15)
clip.write_videofile(output_path, codec='libx264', audio=False)
print(f'Video saved to {output_path}')

clip = ImageSequenceClip(list(depth_images), fps=15)
clip.write_videofile(depth_path, codec='libx264', audio=False)
print(f'Video saved to {depth_path}')

Moviepy - Building video /home/sierra/gaussian-splatting/eval/invivo57/visualization/allcameras.mp4.
Moviepy - Writing video /home/sierra/gaussian-splatting/eval/invivo57/visualization/allcameras.mp4



Moviepy - Done !
Moviepy - video ready /home/sierra/gaussian-splatting/eval/invivo57/visualization/allcameras.mp4
Video saved to /home/sierra/gaussian-splatting/eval/invivo57/visualization/allcameras.mp4
Moviepy - Building video /home/sierra/gaussian-splatting/eval/invivo57/visualization/allcameras_depth.mp4.
Moviepy - Writing video /home/sierra/gaussian-splatting/eval/invivo57/visualization/allcameras_depth.mp4



Moviepy - Done !
Moviepy - video ready /home/sierra/gaussian-splatting/eval/invivo57/visualization/allcameras_depth.mp4
Video saved to /home/sierra/gaussian-splatting/eval/invivo57/visualization/allcameras_depth.mp4


img_array = render_mask_gaussians(model_path, percent_of_gaussians=1, camera_index=novel_view_camera_id)
img = Image.fromarray(img_array, 'RGB')
plt.imshow(img)

images = []

cam_path = os.path.join(model_path, 'cameras.json')
camera_idx_map = load_camera_map(cam_path)

for i in range(num_views):
    camera_idx = camera_idx_map.get(i)
    if camera_idx is not None:
        img_array = render_mask_gaussians(model_path, percent_of_gaussians=i/num_views, camera_index=camera_idx)
        images.append(img_array)

# create video from all frames in full_path using moviepy
clip = ImageSequenceClip(images, fps=15)
clip.write_videofile(output_path, codec='libx264', audio=False)
print(f'Video saved to {output_path}')

spiral = generate_spiral_cam_to_world(radii=[0.1, 0.1, 0.1], focus_depth=center_camera.FoVx, n_poses=120)